In [1]:
import math
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
from random import uniform
from einops.layers.torch import Rearrange
from pytorch_wavelets import DWT, IDWT

import matplotlib.pyplot as plt
from torchvision.transforms import v2
from jdict import jdict

import facer
from facenet_pytorch import MTCNN, InceptionResnetV1

import cv2
from PIL import Image
from insightface.app import FaceAnalysis

import nvdiffrast.torch as dr

import src.shared.util as U 
import src.shared.phong as PH
from src.shared.subdivide import subdivide_trianglemesh

torch.set_printoptions(sci_mode=False, precision=4)

In [2]:
clean = [
    'luke_skywalker_battlefront_2',
    'catherine_hidden_agenda',
    'becky_hidden_agenda',
    'felicity_hidden_agenda',
    'obi_wan_kenobi_battlefront_2',
    'jefferson_bragg_inpatient',
    'jessica_until_dawn',    
    'dante_dmc',
    'sam_until_dawn',
    'amelia_croft_tomb_raider',
    'cassie_drake_uncharted',
    'jonah_maiava_tomb_raider',
    'lara_croft_child_tomb_raider',    
]
minor = [
    'hero_uncharted',
    'lady_dmc',
    'trish_dmc',
    'v_dmc',
    'nico_dmc',
    'morrison_dmc',
    'vergil_dmc',
    'nero_dmc',
    'mike_until_dawn',
    'richard_croft_tomb_raider',
    'elena_fisher_uncharted',
    'abigaile_ortiz_tomb_raider',
    'claire_resident_evil',
    'hank_detroit',
    'connor_detroit',
    'daniel_detroit',
    'chloe_detroit',
    'angela_civilian_detroit',
    'carl_detroit',
    'enchantress_injustice_2',    
]
good_stls = clean + minor
len(good_stls)

33

In [3]:
hp = jdict(    
    device='cuda',    
    renderer_sz=256,    
    camera_position=2.,    
    img_sz=224,
    rndr_sz=512,
    material=0.4,
    color_ratio=0.95,
    color_high=10000,
    direction_ratio=0.9,
    direction_standard=(0.0, 0.0, -1.0),
    eye_x=(-0.2, 0.2),
    eye_y=(-0.2, 0.2),
    eye_z=(2.5, 2.75),
    min_light=0.2,
)

transforms = v2.Compose([
    v2.ToImage(),  # Convert to tensor, only needed if you had a PIL image
    v2.ToDtype(torch.uint8, scale=True),  # optional, most input are already uint8 at this point    
    v2.Resize(size=(224, 224), antialias=True),    
    v2.ToDtype(torch.float32, scale=True),  # Normalize expects float input
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

device = torch.device('cuda')
glctx = dr.RasterizeGLContext()

flip=[1., 1., -1.]
stl_root = './data/face_stls/'
stls = [f for f in sorted(os.listdir(stl_root))
        if (f.endswith('.stl') and f.replace('.stl', '') in good_stls)]

#dinov1_vitb8 = torch.hub.load('facebookresearch/dino:main', 'dino_vitb8')
#dinov2_vitl14 = torch.hub.load('facebookresearch/dinov2', 'dinov2_vitl14')
#dinov2_vitl14_reg = torch.hub.load('facebookresearch/dinov2', 'dinov2_vitl14_reg')

providers=['CUDAExecutionProvider', 'CPUExecutionProvider']
#app_buffalo_l = FaceAnalysis(name="buffalo_l", providers=providers)
#app_buffalo_l.prepare(ctx_id=0, det_size=(640, 640))

app_antelopev2 = FaceAnalysis(name="antelopev2", providers=['CUDAExecutionProvider', 'CPUExecutionProvider'])
app_antelopev2.prepare(ctx_id=0, det_size=(640, 640))

def face_embedding(app, img_file):    
    img = cv2.imread(img_file)
    faces = app.get(img)
    while len(faces) == 0 and app.det_thresh > 0.35:
        app.det_thresh = app.det_thresh - 0.05
        faces = app.get(img)
    app.det_thresh = 0.5
    if faces:
        return torch.from_numpy(faces[0].normed_embedding)
    return None

Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}, 'CUDAExecutionProvider': {'prefer_nhwc': '0', 'enable_skip_layer_norm_strict_mode': '0', 'tunable_op_max_tuning_duration_ms': '0', 'use_ep_level_unified_stream': '0', 'tunable_op_enable': '0', 'enable_cuda_graph': '0', 'cudnn_conv_use_max_workspace': '1', 'do_copy_in_default_stream': '1', 'cudnn_conv_algo_search': 'EXHAUSTIVE', 'gpu_external_empty_cache': '0', 'gpu_external_free': '0', 'tunable_op_tuning_enable': '0', 'cudnn_conv1d_pad_to_nc1d': '0', 'gpu_external_alloc': '0', 'arena_extend_strategy': 'kNextPowerOfTwo', 'has_user_compute_stream': '0', 'gpu_mem_limit': '18446744073709551615', 'device_id': '0'}}
find model: /home/bobi/.insightface/models/antelopev2/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}, 'CUDAExecutionProvider': {'prefer_nhwc': '0',

In [7]:
flip=[1., 1., -1.]
rs = 0.1
n_samples = 100
n_views = 5

face_emb_dir = './data/face_emb_hq/'
tmp_img_file = './render_img.jpg'

image_flip = v2.RandomHorizontalFlip(1)#(render_img)

for i, file in enumerate(stls):
    # if i < 133:
    #     continue
    print(i, file)
    #if i < 
    # if file.replace('.stl', f'_{0}.pth') in os.listdir(face_emb_dir):
    #     continue
    v, f = U.load_mesh(stl_root + file, flip=flip, batch=True)
    while v.size(1) < 3_000:   
        print(v.shape)
        v, f = subdivide_trianglemesh(v.cpu(), f.long(), 1)    
    
    v, faces = v.cuda(), f.int().cuda()
    material = torch.zeros_like(v) + hp.material
    i = 0
    while i < n_samples:        
        scale = torch.Tensor([uniform(1-rs, 1+rs) for _ in range(3)])
        vertices = v * scale.cuda()
        vertices = vertices / vertices.abs().max()
        j, embeddings = 0, []
        while j < n_views:
            render = PH.render_diffuse(glctx, vertices, faces, material, hp)
            render = render.clamp(0, 1)        
            
            render_img = v2.ToPILImage()(render[0].permute(2, 0, 1)) 
            # flipped = bool(randint(0, 1))
            # if flipped:
            #     render_img = image_flip(render_img)
                
            tmp_img_file = face_emb_dir + file.replace('.stl', f'_{i}.jpg')
            render_img.save(tmp_img_file)
    
            # buffalo = face_embedding(app_buffalo_l, tmp_img_file)
            antelope = face_embedding(app_antelopev2, tmp_img_file)
            if antelope is not None:
                embeddings.append(antelope.detach().cpu())  
                j += 1
        torch.save({
            'scale': scale.cpu(),
            #'buffalo': buffalo.detach().cpu(),
            'antelope': embeddings,                    
            'file': file.replace('.stl', ''),
            #'flipped': flipped,
        }, face_emb_dir + file.replace('.stl', f'_{i}.pth'))            
        i += 1
        print(i, file)

0 abigaile_ortiz_tomb_raider.stl
1 abigaile_ortiz_tomb_raider.stl
2 abigaile_ortiz_tomb_raider.stl
3 abigaile_ortiz_tomb_raider.stl
4 abigaile_ortiz_tomb_raider.stl
5 abigaile_ortiz_tomb_raider.stl
6 abigaile_ortiz_tomb_raider.stl
7 abigaile_ortiz_tomb_raider.stl
8 abigaile_ortiz_tomb_raider.stl
9 abigaile_ortiz_tomb_raider.stl
10 abigaile_ortiz_tomb_raider.stl
11 abigaile_ortiz_tomb_raider.stl
12 abigaile_ortiz_tomb_raider.stl
13 abigaile_ortiz_tomb_raider.stl
14 abigaile_ortiz_tomb_raider.stl
15 abigaile_ortiz_tomb_raider.stl
16 abigaile_ortiz_tomb_raider.stl
17 abigaile_ortiz_tomb_raider.stl
18 abigaile_ortiz_tomb_raider.stl
19 abigaile_ortiz_tomb_raider.stl
20 abigaile_ortiz_tomb_raider.stl
21 abigaile_ortiz_tomb_raider.stl
22 abigaile_ortiz_tomb_raider.stl
23 abigaile_ortiz_tomb_raider.stl
24 abigaile_ortiz_tomb_raider.stl
25 abigaile_ortiz_tomb_raider.stl
26 abigaile_ortiz_tomb_raider.stl
27 abigaile_ortiz_tomb_raider.stl
28 abigaile_ortiz_tomb_raider.stl
29 abigaile_ortiz_tomb_r

In [2]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from src.shared.subdivide import Subdivide

src_dir = f'./data/fitted/{512}x{512}/'

# for f in sorted(os.listdir(src_dir)):
#     src = torch.load(src_dir+f)
#     # torch.save(F.adaptive_avg_pool2d(src, 128),
#     #            f'./data/fitted/{128}x{128}/' + f)
#     # torch.save(F.adaptive_avg_pool2d(src, 256),
#     #            f'./data/fitted/{256}x{256}/' + f)
#     for sz in (16, 32, 64):
#         torch.save(F.adaptive_avg_pool2d(src, sz),
#                    f'./data/fitted/{sz}x{sz}/' + f)
    

In [9]:
subdivide = Subdivide()
subdivide.smooth(F.pad(torch.randn(1, 3, 512, 512), (1, 1, 1, 1), mode='replicate')).shape

torch.Size([1, 3, 512, 512])

In [17]:
import src.shared.util as U

for f in sorted(os.listdir(src_dir)):
    src = torch.load(src_dir+f)
    for sz in (256, 128, 64, 32, 16):
        src = F.pad(src, (1, 1, 1, 1), mode='replicate')
        src = subdivide.smooth(src)
        src = F.avg_pool2d(src, 2)
        #print(src.shape, f'./data/fitted/{sz}x{sz}/' + f)
        #U.export_stl(src[0], str(sz))
        torch.save(src, f'./data/fitted/{sz}x{sz}/' + f)    

In [3]:
src_dir = f'./data/fitted/{512}x{512}/'

for f in sorted(os.listdir(src_dir)):
    src = torch.load(src_dir+f)
    for sz in (256, 128, 64, 32, 16):
        src = F.adaptive_avg_pool2d(src, sz)        
        torch.save(src, f'./data/fitted/{sz}x{sz}/' + f)   

In [1]:
import torch
from perlin_cupy import (
    generate_fractal_noise_2d, generate_fractal_noise_3d,
    generate_perlin_noise_2d, generate_perlin_noise_3d
)

In [ ]:
fractal3d = torch.zeros(30_000, 3, 32, 32)
for i in range(30_000):    
    fractal3d[i] = torch.tensor(generate_fractal_noise_3d(
        (4, 32, 32), (2, 2, 2), 2)[:3])
    if i % 100 == 0:
        print(i)
torch.save(fractal3d, 'fractal3d.pth')